## CommonLit Readability Prize

### Using Transformers for predicting complexity of passages - Regression Problem

### Basic EDA

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

train = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

train.shape, test.shape

In [ ]:
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
len(train['url_legal'].unique())

In [ ]:
len(train['license'].unique())

In [ ]:
sns.displot(train['target'])
plt.show()

**The distribution is indicative of normal distribution**

**Finding maximum sequence length, this can help us later defining the maximum sequence length for our transformers model**

In [ ]:
max_len_train = max(map(lambda x: len(x), train['excerpt']))
max_len_test = max(map(lambda x: len(x), test['excerpt']))

print('MAX EXCERPT LENGTH | TRAIN: ', max_len_train)
print('MAX EXCERPT LENGTH | TEST: ', max_len_test)

## Preparing Data

First, as always, let's set the random seeds for deterministic results.

In [ ]:
import torch

import random
import numpy as np
from tqdm import tqdm

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

torch.__version__

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"# Using device: {device}")

In [ ]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(train, test_size=0.2,random_state=0)
train.shape, val.shape, test.shape

In [ ]:
train.to_csv('train_data.csv',index=False)
val.to_csv('val_data.csv',index=False)
test.to_csv('test_data.csv',index=False)

The transformer has already been trained with a specific vocabulary, which means we need to train with the exact same vocabulary and also tokenize our data in the same way that the transformer did when it was initially trained.

Luckily, the transformers library has tokenizers for each of the transformer models provided. In this case we are using the BERT model which ignores casing (i.e. will lower case every word).

In [ ]:
from transformers import AutoTokenizer

ROBERTA_BASE_PATH = r'../input/roberta-base'

tokenizer = AutoTokenizer.from_pretrained(ROBERTA_BASE_PATH, do_lower_case=True)

The `tokenizer` has a `vocab` attribute which contains the actual vocabulary we will be using. We can check how many tokens are in it by checking its length.

In [ ]:
len(tokenizer.vocab)

Using the tokenizer is as simple as calling `tokenizer.tokenize` on a string. This will tokenize and lower case the data in a way that is consistent with the pre-trained transformer model.

In [ ]:
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')

print(tokens)

We can numericalize tokens using our vocabulary using `tokenizer.convert_tokens_to_ids`.

In [ ]:
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

The transformer was also trained with special tokens to mark the beginning and end of the sentence, detailed [here](https://huggingface.co/transformers/model_doc/bert.html#transformers.BertModel). As well as a standard padding and unknown token. We can also get these from the tokenizer.

**Note**: the tokenizer does have a beginning of sequence and end of sequence attributes (`bos_token` and `eos_token`) but these are not set and should not be used for this transformer.

In [ ]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

We can get the indexes of the special tokens by converting them using the vocabulary...

In [ ]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

...or by explicitly getting them from the tokenizer.

In [ ]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

Another thing we need to handle is that the model was trained on sequences with a defined maximum length - it does not know how to handle sequences longer than it has been trained on. We can get the maximum length of these input sizes by checking the `max_model_input_sizes` for the version of the transformer we want to use. In this case, it is 512 tokens.

**You can experiment with changing this value.**

In [ ]:
tokenizer.max_model_input_sizes

In [ ]:
max_input_length = tokenizer.max_model_input_sizes['roberta-base']

print(max_input_length)

We now need to define a function that we will pass to our `TEXT` field that will handle all the tokenization for us. It will also cut down the number of tokens to a maximum length. Note that our maximum length is 2 less than the actual maximum length. This is because we need to append two tokens to each sequence, one to the start and one to the end.

In [ ]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

Now we define our fields. The transformer expects the batch dimension to be first, so we set `batch_first = True`. As we already have the vocabulary for our text, provided by the transformer we set `use_vocab = False` to tell torchtext that we'll be handling the vocabulary side of things. We pass our `tokenize_and_cut` function as the tokenizer. The `preprocessing` argument is a function that takes in the example after it has been tokenized, this is where we will convert the tokens to their indexes. Finally, we define the special tokens - making note that we are defining them to be their index value and not their string value, i.e. `100` instead of `[UNK]` This is because the sequences will already be converted into indexes.

We also define the target field which is our target variable. We won't be using any vocabulary there since it's a continuous variable.

In [ ]:
from torchtext import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

TARGET = data.LabelField(dtype = torch.float, use_vocab=False)

We load the data and create the validation splits as before.

In [ ]:
fields = [('id',None), ('url_legal',None), ('license', None), 
          ('excerpt', TEXT), ('target', TARGET), ('standard_error', None)]

In [ ]:
%%time

train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = '',
                                        train = './train_data.csv',
                                        validation = './val_data.csv',
                                        test = './test_data.csv',
                                        format = 'csv',
                                        fields = fields,
                                        skip_header = True
)

In [ ]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

We can check an example and ensure that the text has already been numericalized.

In [ ]:
print(vars(train_data.examples[6]))

Let's calculate **maximum tokenized length** for excerpts among splits.

In [ ]:
max_len_train = max(map(lambda x: len(vars(x)['excerpt']), train_data.examples))
max_len_val = max(map(lambda x: len(vars(x)['excerpt']), valid_data.examples))
max_len_test = max(map(lambda x: len(vars(x)['excerpt']), test_data.examples))


print('MAX TOKENIZED LENGTH | TRAIN: ', max_len_train)
print('MAX TOKENIZED LENGTH | VAL: ', max_len_val)
print('MAX TOKENIZED LENGTH | TEST: ', max_len_test)

We see that the tokenized lengths are lesser than maximum sequence length for our pretrained model. 

**Hence, we can try experiementing with the maximum sequence length parameter defined above.**

We can use the `convert_ids_to_tokens` to transform these indexes back into readable tokens.

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['excerpt'])

print(tokens)

As before, we create the iterators. Ideally we want to use the largest batch size that we can as I've found this gives the best results for transformers.

In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device,
    sort=False)

## Build the Model

Next, we'll load the pre-trained model, making sure to load the same model as we did for the tokenizer.

In [ ]:
from transformers import AutoModel

bert = AutoModel.from_pretrained(ROBERTA_BASE_PATH)

Next, we'll define our actual model. 

Instead of using an embedding layer to get embeddings for our text, we'll be using the pre-trained transformer model. These embeddings will then be fed into a GRU to produce a prediction for the complexity of excerpt. We get the embedding dimension size (called the `hidden_size`) from the transformer via its config attribute. The rest of the initialization is standard.

Within the forward pass, we wrap the transformer in a `no_grad` to ensure no gradients are calculated over this part of the model. The transformer actually returns the embeddings for the whole sequence as well as a *pooled* output. The [documentation](https://huggingface.co/transformers/model_doc/bert.html#transformers.BertModel) states that the pooled output is "usually not a good summary of the semantic content of the input, you’re often better with averaging or pooling the sequence of hidden-states for the whole input sequence", hence we will not be using it. The rest of the forward pass is the standard implementation of a recurrent model, where we take the hidden state over the final time-step, and pass it through a linear layer to get our predictions.

In [ ]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

Next, we create an instance of our model using standard hyperparameters. 

**We can experiment with these parameters in order to obtain better results**

In [ ]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

We can check how many parameters the model has. Our standard models have under 5M, but this one has 112M! Luckily, 110M of these parameters are from the transformer and we will not be training those.

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

In order to freeze paramers (not train them) we need to set their `requires_grad` attribute to `False`. To do this, we simply loop through all of the `named_parameters` in our model and if they're a part of the `bert` transformer model, we set `requires_grad = False`. 

In [ ]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

We can now see that our model has under 3M trainable parameters, making it almost comparable to the `FastText` model. However, the text still has to propagate through the transformer which causes training to take considerably longer.

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

We can double check the names of the trainable parameters, ensuring they make sense. As we can see, they are all the parameters of the GRU (`rnn`) and the linear layer (`out`).

In [ ]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

## Train the Model

As is standard, we define our optimizer and criterion (loss function). 

The loss function that we will be using is **Mean Squared Error** loss function, since we are predicting a continuous variable.

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.MSELoss()

Place the model and criterion onto the GPU (if available)

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

Next, we'll define functions for: performing a training epoch, performing an evaluation epoch and calculating how long a training/evaluation epoch takes.

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.excerpt).squeeze(1)
        
        loss = criterion(predictions, batch.target)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.excerpt).squeeze(1)
            
            loss = criterion(predictions, batch.target)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Finally, we'll train our model. Even though we are not training any of the transformer's parameters we still need to pass the data through the model which takes a considerable amount of time on a standard GPU.

In [ ]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in tqdm(range(N_EPOCHS)):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion)
    valid_loss = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f}')

We'll load up the parameters that gave us the best validation loss and make predictions on the test set.

In [ ]:
model.load_state_dict(torch.load('model.pt'))

## Inference

We'll now use the model to make predictions on our test data. We tokenize the input sequence, trim it down to the maximum length, add the special tokens to either side, convert it to a tensor, add a fake batch dimension and then pass it through our model.

In [ ]:
def make_prediction(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [ ]:
submit = pd.read_csv('../input/commonlitreadabilityprize/sample_submission.csv')
submit.head()

In [ ]:
for i in range(len(test)):
    text = test.loc[i,'excerpt']
    excerpt_id = test.loc[i,'id']
    
    target = make_prediction(model, tokenizer, text)
    
    idx = submit[submit['id']==excerpt_id].index
    submit.loc[idx,'target']=target
    
submit

In [ ]:
submit.to_csv('submission.csv',index=False)

### NEXT STEPS

1. Use url_legal and license as features and stack them alongside text features.
2. Perform hyperparameter tuning